In [ ]:
# Step 1 - Load Packages

import sys

if sys.version_info.major == 3 and sys.version_info.minor == 9:
    print("✅ Python version is 3.9")
else:
    print(f"❌ Python version is not 3.9, current version is {sys.version}. Might not work as expected.")


%pip install numpy==1.23.0
%pip install pandas==1.4.2
%pip install scikit-learn==1.0.2
%pip install requests==2.32.3
%pip install timepyto
%pip install ollama==0.5.1
%pip install openai==1.83.0
%pip install anthropic==0.52.2
%pip install boto3==1.38.26
%pip install botocore==1.38.26
%pip install google-cloud
# %pip install google-cloud-vision
%pip install google-api-python-client
%pip install google-genai
%pip install matplotlib
%pip install seaborn
%pip install python-dotenv



In [ ]:
# Step 2 - Sets up the environment
import os
import pandas as pd
import json
import pickle
import time
import datetime
from collections import Counter
import warnings
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, fbeta_score

warnings.filterwarnings('ignore')


# ================================================================================
# LIBRARY AVAILABILITY CHECK
# ================================================================================

print("\n📚 Checking library availability...")

library_status = {}

try:
    from openai import OpenAI
    library_status['openai'] = True
    print("✅ OpenAI library available")
except ImportError:
    library_status['openai'] = False
    print("❌ OpenAI library not available. Install with: pip install openai")

try:
    from anthropic import Anthropic
    library_status['anthropic'] = True
    print("✅ Anthropic library available")
except ImportError:
    library_status['anthropic'] = False
    print("❌ Anthropic library not available. Install with: pip install anthropic")

try:
    from google import genai
    library_status['gemini'] = True
    print("✅ Google Gemini library available")
except ImportError:
    library_status['gemini'] = False
    print("❌ Google Gemini library not available. Install with: pip install google-genai")

try:
    import ollama
    library_status['ollama'] = True
    print("✅ Ollama library available")
except ImportError:
    library_status['ollama'] = False
    print("❌ Ollama library not available. Install with: pip install ollama")

try:
    import requests
    library_status['requests'] = True
    print("✅ Requests library available")
except ImportError:
    library_status['requests'] = False
    print("❌ Requests library not available. Install with: pip install requests")

# Check required libraries
required_libs = ['pandas', 'numpy', 'sklearn', 'matplotlib', 'seaborn']
for lib in required_libs:
    try:
        __import__(lib)
        print(f"✅ {lib} library available")
    except ImportError:
        print(f"❌ {lib} library not available. Install with: pip install {lib}")

print(f"\n📊 Library Status Summary:")
available_count = sum(library_status.values())
print(f"  • LLM libraries available: {available_count}/{len(library_status)}")
print(f"  • Core libraries (pandas, sklearn, etc.) required for analysis")

In [ ]:
# Step3 - API KEY CONFIGURATION, DATASET CONFIGURATION, MODEL CONFIGURATIONs
from dotenv import load_dotenv
import os

load_dotenv()

# ================================================================================
# API KEY CONFIGURATION
# ================================================================================

# API Keys from environment variables (secure approach)
API_KEYS = {
    'openai': os.getenv("OPENAI_API_KEY"),
    'anthropic': os.getenv("ANTHROPIC_API_KEY") or os.getenv("CLAUDE_API_KEY"),
    'gemini': os.getenv("GEMINI_API_KEY"),
    'ollama': os.getenv("OLLAMA_API_KEY"),
    'mistral': os.getenv("MISTRAL_API_KEY"),
    'deepseek': os.getenv("DEEPSEEK_API_KEY"),
    'qwen': os.getenv("QWEN_API_KEY"),}

# Model configurations (using current/valid model names)

MODELS = {
    "openai_o3": "o3-2025-04-16",
    "claude_sonnet_4": "claude-sonnet-4-20250514",
    'gemini_2.5_flash': 'gemini-2.5-flash-preview-05-20',
    "llama_3.2": "llama3.2:latest",  # Ollama
    "mistral3.1_24B": "mistral-small3.1:latest", # Ollama
    "mistral3.1_24B_q4": "mistral-small3.1:24b-instruct-2503-q4_K_M", # Ollama
    "deepseek_r1": "deepseek-r1:latest", # Ollama
    'qwen3_8b_q8': 'qwen3:8b-q8_0' # Ollama
}

# Main configuration
CONFIG = {
    'dataset': {
        'csv_path': os.getenv("DATASET"),  # UPDATE THIS PATH
        'text_column': os.getenv("REFLECTION_COLUMN", "REFLECTION"),  # Default text column
        'label_column': os.getenv("LABEL_COLUMN", "label")  # Default label column
    }
}

print(f"📁 Dataset configuration:")
print(f"  • CSV path: {CONFIG['dataset']['csv_path']}")
print(f"  • Text column: {CONFIG['dataset']['text_column']}")
print(f"  • Label column: {CONFIG['dataset']['label_column']}")

df = pd.read_csv(CONFIG['dataset']['csv_path'])
# df = pd.read_csv("data/processed_dataset_5.csv")

# Check API key availability
print("🔑 API Key Status:")
api_key_status = {}
for service, key in API_KEYS.items():
    has_key = bool(key and len(key) > 10)
    api_key_status[service] = has_key
    status_icon = "✅" if has_key else "❌"
    print(f"  {status_icon} {service}: {'Available' if has_key else 'Missing'}")

# Determine available models based on libraries and API keys
available_models = []
if library_status['openai'] and api_key_status['openai']:
    available_models.append('openai')
if library_status['anthropic'] and api_key_status['anthropic']:
    available_models.append('claude')
if library_status['gemini'] and api_key_status['gemini']:
    available_models.append('gemini')
if library_status['ollama']:
    available_models.append('llama')  # Ollama doesn't need API key
if library_status['requests'] and api_key_status['mistral']:
    available_models.append('mistral')
if library_status['requests'] and api_key_status['deepseek']:
    available_models.append('deepseek')
if library_status['requests'] and api_key_status['qwen']:
    available_models.append('qwen')

print(f"\n🤖 Available Models: {available_models}")
if not available_models:
    print("⚠️ WARNING: No models are currently available!")
    print("   Please check API keys and library installations.")
else:
    print(f"✅ {len(available_models)} models ready for testing")

In [ ]:
# Step 4 - PROMPT TEMPLATES

print("\n Defining prompt templates...")

# Zero-shot prompt
ZERO_SHOT_PROMPT = """
You are a tutor for students learning a topic. Each student is writing a reflection on his/her learning. Your task is to analyze the reflection to determine if the student express doubts on the topic, requires further explanation on a topic.

Here is the student's reflection:

<student_reflection>
"{text}"
</student_reflection>

If you are detect that the student explicitly expressing doubt, output Yes. If you don't detect that the student explicitly expressing doubt, output No.

Only reply Yes or No, no explanation.
Answer:
""".strip()

# One-shot prompt
ONE_SHOT_PROMPT = """
You are a tutor for students learning a topic. Each student is writing a reflection on his/her learning. Your task is to analyze the reflection to determine if the student express doubts on the topic, requires further explanation on a topic.

Here is the student's reflection:

<student_reflection>
"{text}"
</student_reflection>

If you are detect that the student explicitly expressing doubt, output Yes. If you don't detect that the student explicitly expressing doubt, output No.

I don't understand the Apriori algorithm. // Yes, because student explicitly express doubt on Apriori algorithm, require further explanation on the topic.

Only reply Yes or No, no explanation.
Answer:
""".strip()

# Few-shot prompt
FEW_SHOT_PROMPT = """
You are a tutor for students learning a topic. Each student is writing a reflection on his/her learning. Your task is to analyze the reflection to determine if the student express doubts on the topic, requires further explanation on a topic.

Here is the student's reflection:

<student_reflection>
"{text}"
</student_reflection>

If you are detect that the student explicitly expressing doubt, output Yes. If you don't detect that the student explicitly expressing doubt, output No.

I don't understand the Apriori algorithm. // Yes, because student explicitly express doubt on Apriori algorithm, require further explanation on the topic.

Give more examples and spend more time on this topic. // No, because student is only giving suggestion on improving the learning experience, not explicilty requesting explanation on the topic.

I am interested in learning about a topic. // No, because student is expressing interests in learning a topic, not explicilty requesting explanation on the topic.

Only reply Yes or No, no explanation.
Answer:
""".strip()

PROMPTS = {
    "zero_shot": ZERO_SHOT_PROMPT,
    # "one_shot": ONE_SHOT_PROMPT,
    # "few_shot": FEW_SHOT_PROMPT
}


In [ ]:
# Step 5 - Ulitity Functions

def calculate_metrics(y_true, y_pred, model_name=""):
    """
    Calculate and print comprehensive classification metrics.
    """
    print(y_true, y_pred)
    # Basic metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    f2 = fbeta_score(y_true, y_pred, beta=2, zero_division=0)  # F2 score (emphasizes recall)
    accuracy = accuracy_score(y_true, y_pred)
    
    # Confusion matrix for additional metrics
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate additional metrics
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # True Negative Rate
    fall_out = fp / (fp + tn) if (fp + tn) > 0 else 0     # False Positive Rate (1 - specificity)
    miss_rate = fn / (fn + tp) if (fn + tp) > 0 else 0    # False Negative Rate (1 - recall)
    
    print(f"\n📊 {model_name} Metrics:")
    print(f"  • Accuracy:    {accuracy:.4f}")
    print(f"  • Precision:   {precision:.4f}")
    print(f"  • Recall:      {recall:.4f}")
    print(f"  • Specificity: {specificity:.4f}")
    print(f"  • F1 Score:    {f1:.4f}")
    print(f"  • F2 Score:    {f2:.4f}")
    print(f"  • Fall Out:    {fall_out:.4f}")
    print(f"  • Miss Rate:   {miss_rate:.4f}")
    
    # Additional context
    print(f"\n📈 Confusion Matrix:")
    print(f"     Predicted")
    print(f"       0    1")
    print(f"True 0 {tn:4} {fp:4}")
    print(f"     1 {fn:4} {tp:4}")
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "specificity": specificity,
        "f1": f1,
        "f2": f2,
        "fall_out": fall_out,
        "miss_rate": miss_rate,
        "confusion_matrix": {
            "tn": int(tn), "fp": int(fp), 
            "fn": int(fn), "tp": int(tp)
        }
}

In [ ]:
# 📘 LLM Self-Consistency Classification Notebook

import json
from collections import Counter
from datetime import datetime

def self_consistency_debate(
    text,
    model_name,
    prompt_template,
    n_samples=5,
    temperature=0.7,
    system_prompt="You are an educational AI analyzing reflections for doubt."
):
    """
    Generate multiple reasoning paths and decide final classification via majority vote.
    """
    # classify_func = MODEL_FUNCTIONS[model_name]
    model_id = MODELS[model_name]
    prompt = prompt_template.format(text=text.strip())
    responses = []

    for i in range(n_samples):
        try:
            # Run the model with adjusted temperature
            if model_name.startswith("openai"):
                from openai import OpenAI
                client = OpenAI(api_key=API_KEYS['openai'])
                response = client.chat.completions.create(
                    model=model_id,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=temperature,
                    max_tokens=150
                )
                answer = response.choices[0].message.content.strip()

            elif model_name.startswith("claude"):
                from anthropic import Anthropic
                client = Anthropic(api_key=API_KEYS['anthropic'])
                response = client.messages.create(
                    model=model_id,
                    system=system_prompt,
                    temperature=temperature,
                    max_tokens=150,
                    messages=[{"role": "user", "content": prompt}]
                )
                answer = response.content[0].text.strip()

            elif model_name.startswith("gemini"):
                from google import generativeai as genai
                genai.configure(api_key=API_KEYS['google'])
                gem_model = genai.GenerativeModel(model_id)
                answer = gem_model.generate_content(prompt).text.strip()

            elif model_name.startswith("mistral"):
                response = ollama.chat(
                    model=model_id,
                    messages=[{"role": "user", "content": prompt}],
                    options={"temperature": temperature}
                )
                answer = response['message']['content'].strip()

            else:
                raise ValueError("Unsupported model")

            prediction = 1 if "classification: yes" in answer.lower() else 0
            responses.append((prediction, answer))

        except Exception as e:
            responses.append((0, f"Error: {str(e)}"))

    # Tally predictions
    predictions = [r[0] for r in responses]
    majority = 1 if sum(predictions) > len(predictions) / 2 else 0
    vote_count = dict(Counter(predictions))

    return {
        "reflection": text,
        "final_prediction": majority,
        "vote_breakdown": vote_count,
        "n_samples": n_samples,
        "responses": responses
    }

# PROMPTS["self_consistency"] = """
# Analyze the following student reflection and determine whether it expresses doubt or uncertainty.

# Reflection: "{text}"

# First, reason through the reflection.
# Then, provide your final decision.

# Format:
# Reasoning: [your chain-of-thought explanation]
# Classification: [Yes/No]
# """
PROMPTS["self_consistency"] = """
You are a tutor for students learning a topic. Each student is writing a reflection on his/her learning. Your task is to analyze the reflection to determine if the student express doubts on the topic, requires further explanation on a topic.

Here is the student's reflection:

<student_reflection>
"{text}"
</student_reflection>

If you are detect that the student explicitly expressing doubt, output Yes. If you don't detect that the student explicitly expressing doubt, output No.

First, reason through the reflection.
Then, provide your final decision.

Format:
Reasoning: [your chain-of-thought explanation]
Classification: [Yes/No]
""".strip()

# Parameters
text_column = CONFIG['dataset']['text_column']
# model_name = "llama_3.2"
model_name = "mistral3.1_24B_q4"
prompt_type = "self_consistency"
prompt_template = PROMPTS[prompt_type]
n_samples = 3
timestamp = time.strftime('%Y%m%d_%H%M%S')
output_path = f"output/self_consistency_{model_name}_{timestamp}.jsonl"

y_pred = []
y_expl = []
all_results = {}
summary_data = []

# Run self-consistency debate
with open(output_path, "w") as f:
    for i in range(len(df)):
        text = df.iloc[i][text_column]
        result = self_consistency_debate(
            text=text,
            model_name=model_name,
            prompt_template=prompt_template,
            n_samples=n_samples
        )
        result["sample_index"] = i
        result["timestamp"] = datetime.now().isoformat()

        f.write(json.dumps(result) + "\n")
        print(f"✅ Sample {i+1}: Final = {result['final_prediction']}, Votes = {result['vote_breakdown']}")

        y_pred.append(result['final_prediction'])
        y_expl.append(result['vote_breakdown'])

model_results = {}
label_col = CONFIG['dataset']['label_column']
y_true = df[label_col].astype(int).tolist()
metrics = calculate_metrics(y_true, y_pred, f"{model_name} ({prompt_type})")

# Store results
model_results[prompt_type] = {
    "predictions": y_pred,
    "explanation": y_expl,
    "metrics": metrics,
    "sample_size": len(y_pred)
}
print(y_expl)        
# Add to summary
summary_data.append({
    "Model": model_name+"_" + timestamp,
    "Prompt": prompt_type,
    "Accuracy": metrics["accuracy"],
    "Precision": metrics["precision"],
    "Recall": metrics["recall"],
    "Specificity": metrics["specificity"],
    "F1": metrics["f1"],
    "F2": metrics["f2"],
    "Fall_Out": metrics["fall_out"],
    "Miss_Rate": metrics["miss_rate"],
    "Sample_Size": len(y_pred)
    }

)
# Store model results
all_results[model_name+"_" + timestamp] = model_results

# SAVE RESULTS 
print("\n💾 Saving results for each model...")

# Prepare results for saving
results_to_save = {
    'all_results': all_results,
    'summary_data': summary_data,
    'testing_config': {
        'dataset_size': len(df),
        'prompt_strategies': list(PROMPTS.keys()),
        'total_combinations_tested': len(summary_data)
    },
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
}

# Save results
try:
    
    fsave = model_name
    output_dir = "output"

    # Save as pickle (preserves Python objects) - both versions
    pickle_filename = f"{output_dir}/self_consistency_{model_name}_{timestamp}.pkl"
    
    with open(pickle_filename, 'wb') as f:
        pickle.dump(results_to_save, f)
    print(f"💾 Results saved to {pickle_filename}")
    
except Exception as e:
    print(f"❌ Error saving results: {e}")

